In [2]:
pip install selenium beautifulsoup4 webdriver-manager pandas

  Using cached selenium-4.33.0-py3-none-any.whl (9.4 MB)
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
  Using cached trio-0.30.0-py3-none-any.whl (499 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
  Using cached exceptiongroup-1.3.0-py3-none-any.whl (16 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.
botocore 1.24.32 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.4.0 which is incompatible.


  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
  Attempting uninstall: certifi
    Found existing installation: certifi 2021.10.8
    Uninstalling certifi-2021.10.8:
      Successfully uninstalled certifi-2021.10.8
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
import time
import pandas as pd


In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://rera.odisha.gov.in/projects/project-list"
driver.get(url)
time.sleep(3)

project_data = []

In [3]:
for i in range(6):
    try:
        print(f"Processing project {i + 1}...")

        view_buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//a[contains(text(),'View Details')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buttons[i])
        driver.execute_script("arguments[0].click();", view_buttons[i])
        time.sleep(4)

        project_name = driver.find_element(By.XPATH, "//label[contains(text(),'Project Name')]/following-sibling::strong").text.strip()
        rera_no = driver.find_element(By.XPATH, "//label[contains(text(),'RERA Regd. No')]/following-sibling::strong").text.strip()

        promoter_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'Promoter Details')]"))
        )
        driver.execute_script("arguments[0].click();", promoter_tab)
        time.sleep(2)

        if i == 2:
            company_name = driver.find_element(By.XPATH, "//label[contains(text(),'Propietory Name')]/following-sibling::strong").text.strip()
            office_address = driver.find_element(By.XPATH, "//label[contains(text(),'Current Residence Address')]/following-sibling::strong").text.strip()
        else:
            company_name = driver.find_element(By.XPATH, "//label[contains(text(),'Company Name')]/following-sibling::strong").text.strip()
            office_address = driver.find_element(By.XPATH, "//label[contains(text(),'Registered Office Address')]/following-sibling::strong").text.strip()

        gst_no = driver.find_element(By.XPATH, "//label[contains(text(),'GST No.')]/following-sibling::strong").text.strip()

        project_data.append({
            "Project Name": project_name,
            "RERA Regd. No.": rera_no,
            "Company Name": company_name,
            "Registered Office Address": office_address,
            "GST No.": gst_no
        })

    except Exception as e:
        print(f"Error in project {i + 1}: {e}")

    try:
        back_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'< Back')]"))
        )
        back_button.click()
    except:
        print("Using < back")
        driver.back()

    time.sleep(3)

Processing project 1...
Using < back
Processing project 2...
Using < back
Processing project 3...
Using < back
Processing project 4...
Using < back
Processing project 5...
Using < back
Processing project 6...
Using < back


In [ ]:
df = pd.DataFrame(project_data)

df.rename(columns={
    "Company Name": "Promoter Name",
    "Registered Office Address": "Address of the Promoter",
    "Current Residence Address": "Address of the Promoter"  # if needed, just in case
}, inplace=True)

print(df)

df.to_csv("odisha_rera_projects.csv", index=False)

driver.quit()

             Project Name    RERA Regd. No.  \
0         Basanti Enclave  RP/01/2025/01362   
1                UDYAYEEN  RP/19/2025/01361   
2  BARSANA RESIDENCY - II  PS/28/2025/01360   
3     KRISHNA MANOR PH-II   PS/7/2025/01358   
4             BHAVYAVILLA  PS/19/2025/01351   
5     MURALIDHARA HEIGHTS  RP/19/2025/01355   

                                     Promoter Name  \
0         M/S. NEELACHAL INFRA DEVELOPERS PVT. LTD   
1              SHYAMCHAND BUILDERS PRIVATE LIMITED   
2                                      RITA PODDAR   
3  KRISHNA PROPERTIES & DEVELOPERS PRIVATE LIMITED   
4                               SUNSHINE INFRATECH   
5                                               --   

                             Address of the Promoter          GST No.  
0  Gurudwara, PO-South Balanda, Via: Talcher Rura...  21AADCN5439J2ZH  
1  MIG-II 21/2 Ground Floor,Chandrasekharpur,Bhub...  21ABCCS4755J1ZB  
2  PLOT NO-2570,PODDAR HEIGHTS,PODDAR COLONY,KHET...  21AREPP3171E1Z7  
3  